# Explore here

# **SISTEMA DE RECOMENDACIÓN DE PELÍCULAS CON MODELO KNN**

In [74]:
# Bibliotecas necesarias

# Biblioteca para manipulación e implementación de datos:
import pandas as pd

# Función para dividir un conjunto de datos en dos subconjuntos (entrenamiento y prueba)
from sklearn.model_selection import train_test_split

# Biblioteca para trabajar con bases de datos SQLite:
import sqlite3

# Clase para convertir una colección de documentos de texto en una matriz de números
from sklearn.feature_extraction.text import CountVectorizer

#
from sklearn.neighbors import KNeighborsClassifier

from sklearn.neighbors import NearestNeighbors

# **Carga de datos**

In [64]:
data_movies = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")
data_credits = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")


In [65]:
data_credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [66]:
data_movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


# **Creación de una base de datos**

- Crea una base de datos para almacenar los dos DataFrames en tablas distintas. 
- A continuación, une las dos tablas con SQL (e intégralo con Python) para generar una tercera tabla que contenga información de ambas unificada. 
- (La clave a través de la cual se puede hacer la unión es el título de la película (`titulo`).)

**Todos estos pasos se realizan porque SQLite es una base de datos ligera y fácil de usar para almacenar datos de forma local**

In [67]:
# Creamos la base de datos llamada: "movies_database"
conn = sqlite3.connect("../data/movies_database.db")

# Creamos las dos tablas en la base de datos
data_movies.to_sql("movies_table", conn, if_exists="replace", index =False)
data_credits.to_sql("credits_table",conn, if_exists = "replace", index = False)

# Unimos las dos tablas generando una tercera
query = """
    SELECT *
    FROM movies_table
    INNER JOIN credits_table
    ON movies_table.title = credits_table.title
"""

# Cargamos los resultados en un dataframe
total_data = pd.read_sql_query(query, conn)

# Cerramos la conexión con la base de datos
conn.close()

total_data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,title,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [68]:
# Eliminamos las columnas que no necesitamos:
columnas_a_conservar = ['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']
data_reducido = total_data[columnas_a_conservar]
data_reducido.head(2)


,movie_id,title,title,overview,genres,keywords,cast,crew
0,19995,Avatar,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [69]:
# Eliminamos una de las columnas "title" para que no aparezcan duplicadas
data_reducido = data_reducido.loc[:, ~data_reducido.columns.duplicated()]

data_reducido.head(2)


,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


# **Transformación de datos**

In [70]:
# Transforma los datos.
# Como puedes ver, hay algunas columnas con formato JSON(Se encuentran entre corchetes o llaves).
# De cada uno de los JSONs, selecciona el atributo `name` y reemplaza las columnas `genres` y `keywords`. Para la columna `cast`, selecciona los tres primeros nombres.

import json

# Primero verificamos el tipo de datos en cada columna.
print(type(data_reducido["genres"].iloc[0]))
print(type(data_reducido["keywords"].iloc[0]))
print(type(data_reducido["cast"].iloc[0]))
print(type(data_reducido["crew"].iloc[0]))





# Procesar la columna "genres" (reemplazar por los valores de "name")
data_reducido['genres'] = data_reducido['genres'].apply(lambda x: [genre['name'] for genre in json.loads(x)])

# Procesar la columna "keywords" (reemplazar por los valores de "name")
data_reducido['keywords'] = data_reducido['keywords'].apply(lambda x: [keyword['name'] for keyword in json.loads(x)])

# Procesar la columna "cast" (seleccionar los 3 primeros nombres)
data_reducido['cast'] = data_reducido['cast'].apply(lambda x: [cast_member['name'] for cast_member in json.loads(x)[:3]])

# Procesar la columna "crew" (puedes especificar algo adicional si es necesario, por ejemplo, directores)
# Aquí seleccionamos solo los nombres de los directores como ejemplo
data_reducido['crew'] = data_reducido['crew'].apply(lambda x: [crew_member['name'] for crew_member in json.loads(x) if crew_member['job'] == 'Director'])

# Mostrar el DataFrame actualizado
print(data_reducido.head())






<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                          genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4   

In [71]:
print(data_reducido.head())


   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                          genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, Science Fiction]   

       

# **Construye un KNN**

In [ ]:
# Para resolver este problema crearemos nosotros nuestro propio KNN. Lo primero de todo es vectorizar el texto siguiendo los mismos pasos que aprendiste en la lección anterior.

# Transformamos todas las columnas (salvo la de "title" y "id") en una lista que contiene el valor en formato cadena (str). 
data_reducido["overview"] = data_reducido["overview"].apply(lambda x: [str(x)])
data_reducido["genres"] = data_reducido["genres"].apply(lambda x: [str(genre) for genre in x])
data_reducido["keywords"] = data_reducido["keywords"].apply(lambda x: [str(keyword) for keyword in x])
data_reducido["cast"] = data_reducido["cast"].apply(lambda x: [str(actor) for actor in x])
data_reducido["crew"] = data_reducido["crew"].apply(lambda x: [str(crew_member) for crew_member in x])


# Combinamos las anteriores columnas en una nueva llamada "tags"
data_reducido["tags"] = data_reducido["overview"] + data_reducido["genres"] + data_reducido["keywords"] + data_reducido["cast"] + data_reducido["crew"]

# Convertimos una lista de palabras clave en una sola cadena de texto y separamos los elementos por espacios.
data_reducido["tags"] = data_reducido["tags"].apply(lambda x: ",".join(x).replace(",", " "))

# Eliminamos columnas utilizadas
data_reducido.drop(columns = ["genres", "keywords", "cast", "crew", "overview"], inplace = True)



data_reducido


,movie_id,title,tags
0,19995,Avatar,In the 22nd century a paraplegic Marine is di...
1,285,Pirates of the Caribbean: At World's End,Captain Barbossa long believed to be dead ha...
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,John Carter is a war-weary former military ca...
...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""Signed Sealed Delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...


In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer



# Convertimos el texto de "tags" en una representación numérica según la importancia de cada palabra.
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data_reducido["tags"])

# Entrenamos el modelo
model = NearestNeighbors(n_neighbors = 6, algorithm = "brute", metric = "cosine")
model.fit(tfidf_matrix)





NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [78]:

def get_movie_recommendations(movie_title):
    movie_index = total_data[total_data["title"] == movie_title].index[0]
    distances, indices = model.kneighbors(tfidf_matrix[movie_index])
    similar_movies = [(total_data["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]

input_movie = "Avatar"
recommendations = get_movie_recommendations(input_movie)
print("Film recommendations '{}'".format(input_movie))
for movie, distance in recommendations:
    print("- Film: {}".format(movie))


ValueError: cannot reindex on an axis with duplicate labels

In [81]:

print(data_reducido.columns.duplicated())  # Devuelve un arreglo indicando duplicados


[False False False]
